In [1]:
# Notebook 03: Procesamiento ETL con Spark
# **Universidad:** Universidad Nacional Experimental de Guayana (UNEG)  
# **Asignatura:** Sistemas de Bases de Datos II  
# **Proyecto:** Proyecto N2 - Data Pipeline Escalable
# ---
# **Descripción:**  
# Extract (Cassandra), Transform (Spark), Load (ClickHouse).

In [2]:
import time
import json
import os
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DecimalType, DateType, IntegerType

# Archivo de métricas compartido (directorio docs montado por Docker)
METRICS_FILE = '../docs/metricas.json'

# Configuración de Spark con conectores de Cassandra y ClickHouse
spark = SparkSession.builder \
    .appName("ETL_Cassandra_Spark_ClickHouse") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0,com.clickhouse:clickhouse-jdbc:0.5.0") \
    .config("spark.cassandra.connection.host", "cassandra") \
    .config("spark.cassandra.connection.localDC", "dc1") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

In [3]:
print("--- 1. Leyendo datos de Cassandra (ventas_db.ventas_crudas) ---")
start_read = time.time()

df_raw = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="ventas_crudas", keyspace="ventas_db") \
    .load()

count_raw = df_raw.count()
end_read = time.time()

print(f"✅ Lectura completada en {end_read - start_read:.2f} segundos")
print(f"Total de registros crudos: {count_raw}")
df_raw.printSchema()
df_raw.show(5)

--- 1. Leyendo datos de Cassandra (ventas_db.ventas_crudas) ---
✅ Lectura completada en 5.85 segundos
Total de registros crudos: 100000
root
 |-- fecha_venta: date (nullable = false)
 |-- id_venta: string (nullable = true)
 |-- cantidad: integer (nullable = true)
 |-- categoria: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- id_producto: string (nullable = true)
 |-- metodo_pago: string (nullable = true)
 |-- monto_total: decimal(38,18) (nullable = true)
 |-- precio_unitario: decimal(38,18) (nullable = true)
 |-- producto: string (nullable = true)
 |-- region: string (nullable = true)

+-----------+--------------------+--------+----------+--------------------+--------------------+---------------+--------------------+--------------------+-----------+------+
|fecha_venta|            id_venta|cantidad| categoria|          id_cliente|         id_producto|    metodo_pago|         monto_total|     precio_unitario|   producto|region|
+-----------+--------------------+

In [4]:
print("--- 2. Transformando datos (Agregación por Fecha y Categoría) ---")
start_transform = time.time()

# Limpieza de datos (Fase 3.3): Filtrar montos nulos o negativos
df_cleaned = df_raw.filter((col("monto_total").isNotNull()) & (col("monto_total") > 0))

# Transformación: Casting, GroupBy, Aggregation
df_aggregated = df_cleaned \
    .withColumn("fecha_dia", col("fecha_venta").cast(DateType())) \
    .groupBy("fecha_dia", "categoria") \
    .agg(
        sum("monto_total").alias("ventas_totales"),
        count("id_venta").alias("cantidad_transacciones")
    )

df_result = df_aggregated.select(
    col("fecha_dia").alias("fecha_venta"),
    col("categoria"),
    col("ventas_totales").cast(DecimalType(18, 2)),
    col("cantidad_transacciones").cast(IntegerType())
)

# Forzamos una acción para medir el tiempo real de transformación (Spark es lazy)
count_result = df_result.count()
end_transform = time.time()

print(f"✅ Transformación completada en {end_transform - start_transform:.2f} segundos")
print(f"Total de filas agregadas after cleaning: {count_result}")
df_result.show(5)

--- 2. Transformando datos (Agregación por Fecha y Categoría) ---
✅ Transformación completada en 3.17 segundos
Total de filas agregadas after cleaning: 5490
+-----------+----------+--------------+----------------------+
|fecha_venta| categoria|ventas_totales|cantidad_transacciones|
+-----------+----------+--------------+----------------------+
| 2024-07-21| Alimentos|       6768.01|                    18|
| 2024-07-21|    Libros|       5280.44|                    14|
| 2024-02-06|     Hogar|       4114.64|                    16|
| 2024-10-31|   Belleza|       3851.69|                    19|
| 2024-10-31|Automotriz|       7153.07|                    16|
+-----------+----------+--------------+----------------------+
only showing top 5 rows



In [5]:
print("--- 3. Escribiendo en ClickHouse (dw_analitico.ventas_resumen) ---")
start_write = time.time()

jdbc_url = "jdbc:clickhouse://clickhouse:8123/dw_analitico"
properties = {
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
}

try:
    df_result.write \
        .mode("append") \
        .jdbc(url=jdbc_url, table="ventas_resumen", properties=properties)
    
    end_write = time.time()
    print(f"✅ Carga en ClickHouse exitosa en {end_write - start_write:.2f} segundos")
except Exception as e:
    end_write = time.time()
    print(f"❌ Error al escribir en ClickHouse: {e}")

--- 3. Escribiendo en ClickHouse (dw_analitico.ventas_resumen) ---
✅ Carga en ClickHouse exitosa en 1.95 segundos


In [6]:
# Calcular tiempos
tiempo_lectura = end_read - start_read
tiempo_transformacion = end_transform - start_transform
tiempo_escritura = end_write - start_write
tiempo_total_etl = tiempo_lectura + tiempo_transformacion + tiempo_escritura

print("--- 📊 Resumen de Métricas de Rendimiento ---")
print(f"1. Lectura (Cassandra):    {tiempo_lectura:.2f} s")
print(f"2. Transformación (Spark): {tiempo_transformacion:.2f} s")
print(f"3. Carga (ClickHouse):     {tiempo_escritura:.2f} s")
print(f"-------------------------------------------")
print(f"Tiempo Total ETL:          {tiempo_total_etl:.2f} s")

--- 📊 Resumen de Métricas de Rendimiento ---
1. Lectura (Cassandra):    5.85 s
2. Transformación (Spark): 3.17 s
3. Carga (ClickHouse):     1.95 s
-------------------------------------------
Tiempo Total ETL:          10.97 s


In [7]:
# =====================================================
# 📊 GUARDAR MÉTRICAS PARA NOTEBOOK 04
# =====================================================

# Leer métricas existentes o crear nuevo diccionario
if os.path.exists(METRICS_FILE):
    with open(METRICS_FILE, 'r') as f:
        metricas = json.load(f)
else:
    metricas = {}

# Actualizar con métricas de este notebook
metricas['etl_spark'] = {
    'tiempo_lectura': round(tiempo_lectura, 2),
    'tiempo_transformacion': round(tiempo_transformacion, 2),
    'tiempo_escritura': round(tiempo_escritura, 2),
    'tiempo_total': round(tiempo_total_etl, 2),
    'registros_entrada': count_raw,
    'registros_salida': count_result,
    'timestamp': datetime.now().isoformat()
}

# Guardar en docs
with open(METRICS_FILE, 'w') as f:
    json.dump(metricas, f, indent=2)

print(f"✅ Métricas guardadas en: {METRICS_FILE}")
print(f"   - Tiempo total ETL: {tiempo_total_etl:.2f} segundos")

✅ Métricas guardadas en: ../docs/metricas.json
   - Tiempo total ETL: 10.97 segundos
